In [1]:
include("./init.jl")

┌ Info: Precompiling Kate [5a7925ad-7fa1-49f4-b2ba-08ed1d7bf936]
└ @ Base loading.jl:1278


Testing environment...
/opt/conda/bin/skewer
/opt/conda/bin/fastqc
/opt/conda/bin/bgzip
/opt/conda/bin/tabix
/opt/conda/bin/minimap2
/opt/conda/bin/samtools
/opt/conda/bin/bcftools
/opt/conda/bin/kallisto
/opt/conda/envs/py2/bin/configManta.py
/opt/conda/envs/py2/bin/configureStrelkaGermlineWorkflow.py
/opt/conda/envs/py2/bin/configureStrelkaSomaticWorkflow.py

Settings are loaded and environment tests passed.


### Set Parameters

In [2]:
par = joinpath(pai, "human_reference_genome")

"/home/jovyan/github/workflow/Kate.jl/input/human_reference_genome"

### Process DNA

In [ ]:
Kate.process_dna(
    se["mo"],
    se["f1"],
    se["f2"],
    se["ad"],
    se["ta"],
    joinpath(pao, string(sa)),
    joinpath(par, "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz"),
    joinpath(par, "chromosome.bed.gz"),
    joinpath(par, "chrn_n.tsv"),
    pasn,
    se["n_jo"],
    se["mt"],
    se["mj"],
)

## Benchmark

TODO: Refactor

In [ ]:
# baseline vcf
vb = joinpath(pai, "high_confidence_HG002_4.1_GRCh38/HG002_GRCh38_1_22_v4.1_draft_benchmark.vcf.gz")

# calls vcf
vc = joinpath(pao, "HG002_cat/find_variant/strelka/results/variants/variants.vcf.gz")

sdf = joinpath(pai, "GRCh38.sdf")

pab = joinpath(pao, "HG002_cat/benchmark")

rtg = "/opt/rtg/rtg-tools-3.11/rtg"

### Configure vcfeval

In [ ]:
run(`$rtg`)

### Download Real Time Genomics prepard GRCH38 files
vcfeval requires reference files in RTG formats. RTG makes versions of these files for the popular reference genomes such as GRCh38, which will be downloaded below.

In [ ]:
if ispath(sdf)
    
    println("Skipping download because RTG GRCh38 data directory already exists:\n $sdf")

else

    run(pipeline(
            `wget -q -P ../input/ https://s3.amazonaws.com/rtg-datasets/references/GRCh38.sdf.zip`,
            `unzip -d ../input/ ../input/GRCh38.sdf.zip`,
            `rm ../input/GRCh38.sdf.zip`,
            ))
end

### Run vcfeval
This command compares the VCF you generated (calls) to the highly confident VCF generated by Genome in a Bottle (baseline).

In [ ]:
run(`$rtg vcfeval
    --baseline=$vb
    --calls=$vc
    --template=$sdf
    --output=$pab
    --threads=1
    `)

### See results
Read more about how to interpret vcfeval results [here](https://cdn.rawgit.com/RealTimeGenomics/rtg-tools/master/installer/resources/tools/RTGOperationsManual/rtg_command_reference.html#vcfeval).

In [ ]:
# vcfeval summary file
pasu = open(joinpath(pab, "summary.txt"))

su = read(pasu)

su